<a href="https://colab.research.google.com/github/Raniamea/arabic-video-summarisation/blob/main/notebooks/01_transcription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# Install Whisper and Torch
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q torch torchaudio --index-url https://download.pytorch.org/whl/cu118



  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.2 MB/s eta 0:00:00


In [3]:
import os
import whisper
import torch

# Paths
input_dir = "/content/drive/MyDrive/ArabicVideoSummariser/videos"
output_dir = "/content/drive/MyDrive/ArabicVideoSummariser/transcripts"

# Create output folder if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load Whisper model
model = whisper.load_model("large", device="cuda")

# Get list of video/audio files
video_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.mp4', '.mp3', '.wav', '.m4a'))]

print(f"Found {len(video_files)} files.")

# Loop through files
for filename in video_files:
    input_path = os.path.join(input_dir, filename)
    base_name = os.path.splitext(filename)[0]
    print(f"\n🔄 Transcribing: {filename} ...")

    try:
        # 1️⃣ Transcribe (Arabic → Arabic)
        result_ar = model.transcribe(input_path, language="ar", task="transcribe")
        transcript_path = os.path.join(output_dir, base_name + "_ar.txt")
        with open(transcript_path, "w", encoding="utf-8") as f:
            f.write(result_ar["text"])
        print(f"✅ Saved Arabic transcript to: {transcript_path}")

        # 2️⃣ Translate (Arabic → English)
        result_en = model.transcribe(input_path, language="ar", task="translate")
        translation_path = os.path.join(output_dir, base_name + "_en.txt")
        with open(translation_path, "w", encoding="utf-8") as f:
            f.write(result_en["text"])
        print(f"✅ Saved English translation to: {translation_path}")

    except Exception as e:
        print(f"❌ Failed to transcribe {filename}: {e}")


100%|█████████████████████████████████████| 2.88G/2.88G [00:33<00:00, 91.6MiB/s]


Found 3 files.

🔄 Transcribing: Calligraphy.mp4 ...
❌ Failed to transcribe Calligraphy.mp4: name 'base_name' is not defined

🔄 Transcribing: KhanElkhalili.mp4 ...
❌ Failed to transcribe KhanElkhalili.mp4: name 'base_name' is not defined

🔄 Transcribing: Upholstery.mp4 ...


KeyboardInterrupt: 